In [ ]:
# Install PyTorch (specific version may depend on your system's CUDA compatibility)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install Transformers and Diffusers
!pip install transformers diffusers

# Install Flask
!pip install Flask

# Install pyngrok
!pip install pyngrok

# Install Pillow
!pip install Pillow

# Optional: Install Google Colab library (if running in Google Colab)
!pip install google-colab
!pip install transformers diffusers

# Install Flask
!pip install Flask

# Install pyngrok
!pip install pyngrok

# Install Pillow
!pip install Pillow

# Optional: Install Google Colab library (if running in Google Colab)
!pip install google-colab
!pip install --quiet xformers
!pip install --quiet diffusers transformers accelerate


In [15]:
from pyngrok import ngrok
from flask import Flask, request, render_template_string, send_from_directory, redirect, url_for, render_template
from diffusers import StableDiffusionPipeline
import torch
from google.colab import drive
from PIL import Image, ImageDraw, ImageFont
import os
import textwrap
import shutil

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set up ngrok token (use your actual token here)
ngrok.set_auth_token("2rXWODUjvMzp8SMPms27EWBY9z1_26Vn2dFjfPF9gzW2W35V6")  # Replace with your actual ngrok token

# Step 3: Load the Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"  # Replace with any anime-focused model if needed
pipe = StableDiffusionPipeline.from_pretrained(model_id)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")
else:
    print("CUDA not available. Running on CPU. This might be slower.")

# Step 4: Create a Flask app
app = Flask(__name__)

# Step 5: Static file paths for images and font
static_folder = "/content/static"
os.makedirs(static_folder, exist_ok=True)

@app.route('/static/<path:filename>')
def static_files(filename):
    return send_from_directory(static_folder, filename)

@app.route("/contact")
def contact():
    # Render the external HTML template
    with open("/content/static/contact_us.html", "r") as html_file:
        return render_template_string(html_file.read())

@app.route("/generate-comic", methods=["GET", "POST"])
def generate_comic():
    if request.method == "POST":
        # Retrieve form data
        num_slides = int(request.form.get("num_slides"))
        character_desc = request.form.get("character_desc")
        scene_descs = request.form.getlist("scene_desc[]")
        dialogues = request.form.getlist("dialogue[]")

        # Validate inputs
        if len(scene_descs) != num_slides or len(dialogues) != num_slides:
            return "Error: Number of scenes and dialogues must match the number of panels.", 400

        # Directory for generated panels
        panel_urls = []
        for i in range(num_slides):
            prompt = f"{character_desc}. Scene: {scene_descs[i]}. Dialogue: {dialogues[i]}"
            image = pipe(prompt).images[0]
            output_path = os.path.join(static_folder, f"comic_panel_{i + 1}.jpg")
            image.save(output_path, format="JPEG")
            panel_urls.append(f"/static/comic_panel_{i + 1}.jpg")  # Save the panel URL

        # Pass panel URLs to view_comic.html
        return render_comic_page(panel_urls)

    # Render the external HTML template
    return render_generate_comic_page()


def render_comic_page(panel_urls):
    # Read the view_comic.html from the static folder
    with open(os.path.join('static', 'view_comic.html'), 'r') as file:
        return render_template_string(file.read(), panel_urls=panel_urls)


def render_generate_comic_page():
    # Read the generate_comic.html from the static folder
    with open(os.path.join('static', 'generate_comic.html'), 'r') as file:
        return render_template_string(file.read())


@app.route("/view-comic")
def view_comic():
    comic_url = "/static/comic_strip.jpg"  # Path to the generated comic image
    download_url = url_for('download_comic')  # Path to the download route
    return render_comic_page([comic_url])  # Modify as needed to match the structure


@app.route("/download-comic")
def download_comic():
    comic_path = os.path.join(static_folder, "comic_strip.jpg")
    if not os.path.exists(comic_path):
        return "Error: Comic strip file not found. Please generate the comic first.", 404
    return send_file(comic_path, as_attachment=True, mimetype='image/jpeg', download_name="comic_strip.jpg")

@app.route("/about")
def about():
    # Render the external HTML template
    with open("/content/static/about_us.html", "r") as html_file:
        return render_template_string(html_file.read())

@app.route("/", methods=["GET", "POST"])
def generate_image():
    image_path = None
    if request.method == "POST":
        prompt = request.form["prompt"]
        print(f"Generating image for prompt: {prompt}")

        # Generate the image
        image = pipe(prompt).images[0]

        # Save the image in JPG format to a temporary directory
        output_dir = "/content/Generated_Comics"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "comic_slide.jpg")
        image.save(output_path, format="JPEG")

        # Make it accessible via the web
        image_path = f"/static/comic_slide.jpg"
        shutil.copy(output_path, os.path.join(static_folder, "comic_slide.jpg"))

    # Render the external HTML template
    with open("/content/static/maintemplate.html", "r") as html_file:
        return render_template_string(html_file.read(), image_path=image_path)

# Create ngrok tunnel and run the app
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

if __name__ == "__main__":
    app.run(port=5000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * ngrok tunnel "NgrokTunnel: "https://2cbc-34-75-2-47.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:02] "GET /static/logo%20cgai%202.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:02] "GET /static/bg1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:02] "GET /static/Comic%20Book.otf HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:05] "GET /generate-comic HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:05] "GET /static/logo%20cgai%202.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:25:05] "GET /static/bg1.jpg HTTP/1.1" 304 -
Token indices sequence length is longer than the specified maximum sequence 

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:12] "POST /generate-comic HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:12] "GET /static/logo%20cgai%202.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:12] "GET /static/comic_panel_1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:13] "GET /static/bg1.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:24] "GET /generate-comic HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:24] "GET /static/bg1.jpg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [16/Jan/2025 08:26:24] "GET /static/logo%20cgai%202.jpg HTTP/1.1" 304 -


In [2]:
# Updated HTML template with logo implementation
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Comix GenAI</title>
    <style>
        @font-face {
            font-family: 'ComicBook';
            src: url('/static/Comic Book.otf') format('truetype');
        }

       body {
            font-family: 'ComicBook', sans-serif;
            background-color: #f8f8f8;
            color: #333;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            background-image: url('/static/bg1.jpg');
            background-size: cover;
            background-position: center;
        }

        .sidebar {
            width: 60px;
            height: 100vh;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding-top: 20px;
            position: fixed;
            left: 0;
            transition: width 0.3s ease;
            overflow: hidden;
            box-shadow: 5px 0 15px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover, .sidebar:focus-within {
            width: 250px;
        }

        .sidebar .logo {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            margin-bottom: 20px;
            padding: 0 10px;
        }

        .sidebar .logo img {
            width: 60px;
            height: 60px;
            margin-right: 10px;
        }

        .sidebar .logo h1 {
            font-size: 24px;
            color: #ff5733;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            white-space: nowrap;
        }

        .sidebar button {
            width: 100%;
            padding: 12px;
            margin: 12px 0;
            background-color: #ff5733;
            border: none;
            color: white;
            text-align: center;
            font-size: 18px;
            cursor: pointer;
            border-radius: 10px;
            transition: all 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }

        .sidebar button:hover {
            background-color: #c93c22;
        }

        .sidebar button span {
            display: inline-block;
            opacity: 0;
            transition: opacity 0.3s ease;
            padding-left: 10px;
        }

        .sidebar:hover button span {
            opacity: 1;
        }

        .main {
            margin-left: 60px;
            padding: 20px;
            width: calc(100% - 60px);
            transition: margin-left 0.3s ease;
            font-family: 'ComicBook', sans-serif;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover ~ .main {
            margin-left: 250px;
            width: calc(100% - 250px);
        }

        .navbar {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding: 12px 20px;
            position: sticky;
            top: 0;
            z-index: 1;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            font-family: 'ComicBook', sans-serif;
            border-radius: 10px;
            margin-bottom: 20px;
        }

        .navbar .nav-links {
            display: flex;
            margin-left: 0;
        }

        .navbar a {
            color: white;
            text-decoration: none;
            padding: 12px 20px;
            font-size: 18px;
            transition: background-color 0.3s ease;
        }

        .navbar a:hover {
            background-color: #ff5733;
            border-radius: 10px;
        }

        form {
            display: inline-block;
            margin-top: 20px;
        }

        form input {
            width: 100%;
            padding: 12px;
            font-size: 16px;
            border: 2px solid #ff5733;
            border-radius: 8px;
            background-color: #fff;
            font-family: 'ComicBook', sans-serif;
        }

        form button {
            padding: 12px 24px;
            font-size: 16px;
            background-color: #ff5733;
            color: white;
            border: none;
            border-radius: 10px;
            cursor: pointer;
            transition: background-color 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }

        form button:hover {
            background-color: #c93c22;
        }

        h2, h3 {
            font-size: 36px;
            color: #222;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            font-family: 'ComicBook', sans-serif;
        }

        h3 {
            font-size: 28px;
            margin-top: 20px;
        }

        .generated-image {
            border: 5px dashed #ff5733;
            padding: 10px;
            margin-top: 20px;
            display: inline-block;
            background-color: #f3f3f3;
            border-radius: 10px;
        }

        a {
            text-decoration: none;
            color: #ff5733;
            font-weight: bold;
        }

        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <div class="logo">
            <img src="/static/logo cgai 2.jpg" alt="Logo">
            <h1>Comix GenAI</h1>
        </div>
        <button onclick="location.href='/generate-story'"><span>Generate a Story</span></button>
        <button onclick="location.href='/generate-images'"><span>Generate Images</span></button>
        <button onclick="location.href='/generate-comic'"><span>Generate Comic</span></button>
        <button onclick="location.href='/image-colorization'"><span>Image Colorization</span></button>
    </div>

    <div class="main">
        <div class="navbar">
            <div class="nav-links">
                <a href="/">Home</a>
                <a href="/history">History</a>
                <a href="/about">About Us</a>
                <a href="/contact">Contact Us</a>
            </div>
        </div>

        <h2>Generate Your Comic-Style Image</h2>
        <form action="/" method="post">
            <label for="prompt">Enter your prompt:</label><br><br>
            <input type="text" id="prompt" name="prompt" placeholder="E.g., A gang fight in Tokyo Revengers style" required><br><br>
            <button type="submit">Generate Image</button>
        </form>
        {% if image_path %}
        <div class="generated-image">
            <h3>Generated Image:</h3>
            <img src="{{ image_path }}" width="512px">
            <p><a href="{{ image_path }}" download>Download Image</a></p>
        </div>
        {% endif %}
    </div>
</body>
</html>
"""
# Save the HTML file
with open("/content/static/maintemplate.html", "w") as html_file:
    html_file.write(HTML_TEMPLATE)


In [3]:

# About page template
ABOUT_US_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>About Us</title>
    <style>
        body {
            font-family: 'ComicBook', sans-serif;
            background-color: #f8f8f8;
            color: #333;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            background-image: url('/static/bg1.jpg');
            background-size: cover;
            background-position: center;
        }
        h2 {
            color: #333;
            font-size: 36px;
            margin-bottom: 10px;
            border-bottom: 2px solid #ccc;
            padding-bottom: 10px;
        }
        h3 {
            color: #333;
            font-size: 28px;
            margin-top: 40px;
        }
        p {
            font-size: 18px;
            line-height: 1.6;
        }
        ul {
            list-style-type: disc;
            padding-left: 40px;
        }
        li {
            margin: 5px 0;
        }
        .team-members {
            display: flex;
            justify-content: space-around;
            flex-wrap: wrap;
            gap: 20px;
            margin-top: 20px;
        }

        .sidebar {
            width: 60px;
            height: 100vh;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding-top: 20px;
            position: fixed;
            left: 0;
            transition: width 0.3s ease;
            overflow: hidden;
            box-shadow: 5px 0 15px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover, .sidebar:focus-within {
            width: 250px;
        }

        .sidebar .logo {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            margin-bottom: 20px;
            padding: 0 10px;
        }

        .sidebar .logo img {
            width: 60px;
            height: 60px;
            margin-right: 10px;
        }

        .sidebar .logo h1 {
            font-size: 24px;
            color: #ff5733;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            white-space: nowrap;
        }

        .sidebar button {
            width: 100%;
            padding: 12px;
            margin: 12px 0;
            background-color: #ff5733;
            border: none;
            color: white;
            text-align: center;
            font-size: 18px;
            cursor: pointer;
            border-radius: 10px;
            transition: all 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }

        .sidebar button:hover {
            background-color: #c93c22;
        }

        .sidebar button span {
            display: inline-block;
            opacity: 0;
            transition: opacity 0.3s ease;
            padding-left: 10px;
        }

        .sidebar:hover button span {
            opacity: 1;
        }

        .main {
            margin-left: 60px;
            padding: 20px;
            width: calc(100% - 60px);
            transition: margin-left 0.3s ease;
            font-family: 'ComicBook', sans-serif;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover ~ .main {
            margin-left: 250px;
            width: calc(100% - 250px);
        }

        .navbar {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding: 12px 20px;
            position: sticky;
            top: 0;
            z-index: 1;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            font-family: 'ComicBook', sans-serif;
            border-radius: 10px;
            margin-bottom: 20px;
        }

        .navbar .nav-links {
            display: flex;
            margin-left: 0;
        }

        .navbar a {
            color: white;
            text-decoration: none;
            padding: 12px 20px;
            font-size: 18px;
            transition: background-color 0.3s ease;
        }

        .navbar a:hover {
            background-color: #ff5733;
            border-radius: 10px;
        }

        .team-member {
            text-align: center;
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            width: 250px;
        }
        .team-member img {
            width: 100px;
            height: 100px;
            border-radius: 50%;
            margin-bottom: 10px;
        }
        .team-member h3 {
            font-size: 24px;
            margin: 10px 0;
            color: #333;
        }
        .team-member p {
            font-size: 16px;
            color: #666;
            margin: 5px 0;
        }
        .flowchart img {
            width: 200px; /* Flowchart width */
            height: 250px; /* Flowchart height */
            border: 1px solid #ccc;
            border-radius: 8px;
            margin-top: 20px;
        }
        .inspiration {
            margin-top: 40px;
            font-size: 18px;
            line-height: 1.6;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <div class="logo">
            <img src="/static/logo cgai 2.jpg" alt="Logo">
            <h1>Comix GenAI</h1>
        </div>
        <button onclick="location.href='/generate-story'"><span>Generate a Story</span></button>
        <button onclick="location.href='/generate-images'"><span>Generate Images</span></button>
        <button onclick="location.href='/generate-comic'"><span>Generate Comic</span></button>
        <button onclick="location.href='/image-colorization'"><span>Image Colorization</span></button>
    </div>

    <div class="main">
        <div class="navbar">
            <div class="nav-links">
                <a href="/">Home</a>
                <a href="/history">History</a>
                <a href="/about">About Us</a>
                <a href="/contact">Contact Us</a>
            </div>
        </div>
    <p>Welcome to <strong>Comix GenAI</strong>, a platform that brings your creative ideas to life with AI-generated comic-style images.</p>

    <h3>Introduction</h3>
    <ul>
        <li>ComixGen AI is an AI tool that turns story prompts into comic strips.</li>
        <li>Users provide a story idea, and the system automatically generates comic panels with images and captions.</li>
        <li>It uses Stable Diffusion for image generation and Flask to create a web interface for interaction.</li>
        <li>The tool is made accessible globally through ngrok, which exposes the local server to the web.</li>
        <li>It simplifies comic creation, making it fun and great for storytelling, education, and creative projects.</li>
    </ul>

    <h3>Features</h3>
    <ul>
        <li>Generate comic-style images from text prompts.</li>
        <li>Colorize existing black-and-white images.</li>
        <li>Create custom comic stories.</li>
    </ul>

    <h3>Inspiration</h3>
    <div class="inspiration">
        <p><strong>Make Creativity Easy for Everyone:</strong> We wanted to help people create comics without needing drawing skills.</p>
        <p><strong>Make Learning Fun:</strong> We aim to explain tough topics with pictures and stories that are easy to understand.</p>
        <p><strong>Include Everyone:</strong> Our goal is to give everyone, including those with disabilities, a way to share their ideas through comics.</p>
    </div>

    <h3>Team Members</h3>
    <div class="team-members">
        <div class="team-member">
            <img src="/static/jyoti.jpg" alt="Jyoti Singh">
            <h3>Jyoti Singh</h3>
            <p>Email: jyotisingh190402@gmail.com</p>
            <p>Phone: +91 85301 04001</p>
            <p><a href="https://www.linkedin.com/in/jyotisingh190402/" target="_blank">LinkedIn</a></p>
        </div>
        <div class="team-member">
            <img src="/static/hitakshi.jpg" alt="Hitakshi Trivedi">
            <h3>Hitakshi Trivedi</h3>
            <p>Email: hitutrivedi9@gmail.com</p>
            <p>Phone: +91 97663 78853</p>
            <p><a href="https://www.linkedin.com/in/hitakshi-trivedi-460a1127b/" target="_blank">LinkedIn</a></p>
        </div>
        <div class="team-member">
            <img src="/static/aditya.jpg" alt="Aditya Singh">
            <h3>Aditya Singh</h3>
            <p>Email: singhadityag1006@gmail.com</p>
            <p>Phone: +91 80805 03530</p>
            <p><a href="https://www.linkedin.com/in/aditya-singh-415a69294/" target="_blank">LinkedIn</a></p>
        </div>
    </div>
</body>
</html>
"""
# Save the HTML file
with open("/content/static/about_us.html", "w") as html_file:
    html_file.write(ABOUT_US_TEMPLATE)

In [4]:
# Contact page template
contact_us_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Contact Us</title>
    <style>
        body {
            font-family: 'ComicBook', sans-serif;
            background-color: #f8f8f8;
            color: #333;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            background-image: url('/static/bg1.jpg');
            background-size: cover;
            background-position: center;
        }

        h2 {
            color: #333;
        }

        p {
            color: #666;
            font-size: 1rem;
        }


        .sidebar {
            width: 60px;
            height: 100vh;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding-top: 20px;
            position: fixed;
            left: 0;
            transition: width 0.3s ease;
            overflow: hidden;
            box-shadow: 5px 0 15px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover, .sidebar:focus-within {
            width: 250px;
        }

        .sidebar .logo {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            margin-bottom: 20px;
            padding: 0 10px;
        }

        .sidebar .logo img {
            width: 60px;
            height: 60px;
            margin-right: 10px;
        }

        .sidebar .logo h1 {
            font-size: 24px;
            color: #ff5733;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            white-space: nowrap;
        }

        .sidebar button {
            width: 100%;
            padding: 12px;
            margin: 12px 0;
            background-color: #ff5733;
            border: none;
            color: white;
            text-align: center;
            font-size: 18px;
            cursor: pointer;
            border-radius: 10px;
            transition: all 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }

        .sidebar button:hover {
            background-color: #c93c22;
        }

        .sidebar button span {
            display: inline-block;
            opacity: 0;
            transition: opacity 0.3s ease;
            padding-left: 10px;
        }

        .sidebar:hover button span {
            opacity: 1;
        }

        .main {
            margin-left: 60px;
            padding: 20px;
            width: calc(100% - 60px);
            transition: margin-left 0.3s ease;
            font-family: 'ComicBook', sans-serif;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover ~ .main {
            margin-left: 250px;
            width: calc(100% - 250px);
        }

        .navbar {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding: 12px 20px;
            position: sticky;
            top: 0;
            z-index: 1;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            font-family: 'ComicBook', sans-serif;
            border-radius: 10px;
            margin-bottom: 20px;
        }

        .navbar .nav-links {
            display: flex;
            margin-left: 0;
        }

        .navbar a {
            color: white;
            text-decoration: none;
            padding: 12px 20px;
            font-size: 18px;
            transition: background-color 0.3s ease;
        }

        .navbar a:hover {
            background-color: #ff5733;
            border-radius: 10px;
        }

        .team-container {
            display: flex;
            justify-content: center;
            gap: 50px; /* Increased space between members */
            flex-wrap: wrap;
            margin-top: 20px;
        }

        .member {
            width: 300px; /* Increased size */
            height: 300px; /* Increased size */
            border-radius: 50%;
            background-color: white;
            border: 3px solid black; /* Black border added */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            padding: 20px;
            box-sizing: border-box;
            position: relative;
        }

        .member img {
            width: 100px;
            height: 100px;
            border-radius: 50%;
            margin-bottom: 10px;
        }

        .member strong {
            font-size: 1.2rem;
            color: #333;
        }

        .member span {
            font-size: 0.9rem;
            color: #555;
        }

        .member a {
            color: #007BFF;
            text-decoration: none;
            font-size: 0.9rem;
            display: block;
            margin-top: 5px;
        }

        .member a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <div class="logo">
            <img src="/static/logo cgai 2.jpg" alt="Logo">
            <h1>Comix GenAI</h1>
        </div>
        <button onclick="location.href='/generate-story'"><span>Generate a Story</span></button>
        <button onclick="location.href='/generate-images'"><span>Generate Images</span></button>
        <button onclick="location.href='/generate-comic'"><span>Generate Comic</span></button>
        <button onclick="location.href='/image-colorization'"><span>Image Colorization</span></button>
    </div>

    <div class="main">
        <div class="navbar">
            <div class="nav-links">
                <a href="/">Home</a>
                <a href="/history">History</a>
                <a href="/about">About Us</a>
                <a href="/contact">Contact Us</a>
            </div>
        </div>
    <p>For any inquiries, feel free to reach out to our team members below:</p>

    <div class="team-container">
        <!-- Member 1 -->
        <div class="member">
            <img src="/static/jyoti.jpg" alt="Jyoti Singh">
            <strong>Jyoti Singh</strong>
            <span>Email:</span>
            <a href="mailto:jyotisingh190402@gmail.com">jyotisingh190402@gmail.com</a>
            <span>Phone:</span>
            <span>+91 85301 04001</span>
        </div>

        <!-- Member 2 -->
        <div class="member">
            <img src="/static/hitakshi.jpg" alt="Hitakshi Trivedi">
            <strong>Hitakshi Trivedi</strong>
            <span>Email:</span>
            <a href="mailto:hitudrivedi9@gmail.com">hitudrivedi9@gmail.com</a>
            <span>Phone:</span>
            <span>+91 97663 78853</span>
        </div>

        <!-- Member 3 -->
        <div class="member">
            <img src="/static/aditya.jpg" alt="Aditya Singh">
            <strong>Aditya Singh</strong>
            <span>Email:</span>
            <a href="mailto:singhadityag1006@gmail.com">singhadityag1006@gmail.com</a>
            <span>Phone:</span>
            <span>+91 80805 03530</span>
        </div>
    </div>
</body>
</html>
"""
# Save the HTML file
with open("/content/static/contact_us.html", "w") as html_file:
    html_file.write(contact_us_html)

In [5]:
generate_comic_html ="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>generate_comic</title>
    <style>
        body {
            font-family: 'ComicBook', sans-serif;
            background-color: #f8f8f8;
            color: #333;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            background-image: url('/static/bg1.jpg');
            background-size: cover;
            background-position: center;
        }

        h2 {
            color: #333;
        }

        p {
            color: #666;
            font-size: 1rem;
        }

        .sidebar {
            width: 60px;
            height: 100vh;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding-top: 20px;
            position: fixed;
            left: 0;
            transition: width 0.3s ease;
            overflow: hidden;
            box-shadow: 5px 0 15px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover, .sidebar:focus-within {
            width: 250px;
        }

        .sidebar .logo {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            margin-bottom: 20px;
            padding: 0 10px;
        }

        .sidebar .logo img {
            width: 60px;
            height: 60px;
            margin-right: 10px;
        }

        .sidebar .logo h1 {
            font-size: 24px;
            color: #ff5733;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            white-space: nowrap;
        }

        .sidebar button {
            width: 100%;
            padding: 12px;
            margin: 12px 0;
            background-color: #ff5733;
            border: none;
            color: white;
            text-align: center;
            font-size: 18px;
            cursor: pointer;
            border-radius: 10px;
            transition: all 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }

        .sidebar button:hover {
            background-color: #c93c22;
        }

        .sidebar button span {
            display: inline-block;
            opacity: 0;
            transition: opacity 0.3s ease;
            padding-left: 10px;
        }

        .sidebar:hover button span {
            opacity: 1;
        }

        .main {
            margin-left: 60px;
            padding: 20px;
            width: calc(100% - 60px);
            transition: margin-left 0.3s ease;
            font-family: 'ComicBook', sans-serif;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2);
        }

        .sidebar:hover ~ .main {
            margin-left: 250px;
            width: calc(100% - 250px);
        }

        .navbar {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding: 12px 20px;
            position: sticky;
            top: 0;
            z-index: 1;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            font-family: 'ComicBook', sans-serif;
            border-radius: 10px;
            margin-bottom: 20px;
        }

        .navbar .nav-links {
            display: flex;
            margin-left: 0;
        }

        .navbar a {
            color: white;
            text-decoration: none;
            padding: 12px 20px;
            font-size: 18px;
            transition: background-color 0.3s ease;
        }

        .navbar a:hover {
            background-color: #ff5733;
            border-radius: 10px;
        }

        .form-container {
            margin: 20px auto;
            max-width: 800px;
            padding: 20px;
            background-color: rgba(255, 255, 255, 0.8);
            border-radius: 10px;
            box-shadow: 0 0 15px rgba(0, 0, 0, 0.2);
        }

        .form-container label {
            font-size: 16px;
            font-weight: bold;
        }

        .form-container textarea, .form-container input {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border: 1px solid #ccc;
            border-radius: 5px;
        }

        .form-container button {
            background-color: #ff5733;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
            transition: background-color 0.3s ease;
        }

        .form-container button:hover {
            background-color: #c93c22;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <div class="logo">
            <img src="/static/logo cgai 2.jpg" alt="Logo">
            <h1>Comix GenAI</h1>
        </div>
        <button onclick="location.href='/generate-story'"><span>Generate a Story</span></button>
        <button onclick="location.href='/generate-images'"><span>Generate Images</span></button>
        <button onclick="location.href='/generate-comic'"><span>Generate Comic</span></button>
        <button onclick="location.href='/image-colorization'"><span>Image Colorization</span></button>
    </div>

    <div class="main">
        <div class="navbar">
            <div class="nav-links">
                <a href="/">Home</a>
                <a href="/history">History</a>
                <a href="/about">About Us</a>
                <a href="/contact">Contact Us</a>
            </div>
        </div>

        <div class="form-container">
            <h2>Generate Your Comic</h2>
            <form method="POST" action="/generate-comic">
                <label for="num_slides">Number of Panels:</label>
                <input type="number" id="num_slides" name="num_slides" min="1" max="10" required>

                <label for="character_desc">Character Description:</label>
                <textarea id="character_desc" name="character_desc" rows="4" required></textarea>

                <div id="panels">
                    <div>
                        <h3>Panel 1:</h3>
                        <label for="scene_desc_1">Scene Description:</label>
                        <textarea id="scene_desc_1" name="scene_desc[]" rows="3" required></textarea>

                        <label for="dialogue_1">Dialogue:</label>
                        <textarea id="dialogue_1" name="dialogue[]" rows="2" required></textarea>
                    </div>
                </div>

                <button type="button" onclick="addPanel()">Add Another Panel</button>
                <button type="submit">Generate Comic</button>
            </form>
        </div>
    </div>

    <script>
        let panelCount = 1;
        function addPanel() {
            panelCount++;
            const panelDiv = document.createElement('div');
            panelDiv.innerHTML = `
                <h3>Panel ${panelCount}:</h3>
                <label for="scene_desc_${panelCount}">Scene Description:</label>
                <textarea id="scene_desc_${panelCount}" name="scene_desc[]" rows="3" required></textarea>

                <label for="dialogue_${panelCount}">Dialogue:</label>
                <textarea id="dialogue_${panelCount}" name="dialogue[]" rows="2" required></textarea>
            `;
            document.getElementById('panels').appendChild(panelDiv);
        }
    </script>
</body>
</html>
"""

# Save the HTML file
with open("/content/static/generate_comic.html", "w") as html_file:
    html_file.write(generate_comic_html)

In [11]:
view_comic_html ="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Your Comic</title>
    <style>
        body {
            font-family: 'ComicBook', sans-serif;
            background-color: #f8f8f8;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            background-image: url('/static/bg1.jpg');
            background-size: cover;
            background-position: center;
        }
        .sidebar {
            width: 60px;
            height: 100vh;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding-top: 20px;
            position: fixed;
            left: 0;
            transition: width 0.3s ease;
            overflow: hidden;
            box-shadow: 5px 0 15px rgba(0, 0, 0, 0.2);
        }
        .sidebar:hover {
            width: 250px;
        }
        .sidebar .logo {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            margin-bottom: 20px;
            padding: 0 10px;
        }
        .sidebar .logo img {
            width: 60px;
            height: 60px;
            margin-right: 10px;
        }
        .sidebar .logo h1 {
            font-size: 24px;
            color: #ff5733;
            text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
            white-space: nowrap;
        }
        .sidebar button {
            width: 100%;
            padding: 12px;
            margin: 12px 0;
            background-color: #ff5733;
            border: none;
            color: white;
            text-align: center;
            font-size: 18px;
            cursor: pointer;
            border-radius: 10px;
            transition: all 0.3s ease;
            font-family: 'ComicBook', sans-serif;
        }
        .sidebar button:hover {
            background-color: #c93c22;
        }
        .main {
            margin-left: 60px;
            padding: 20px;
            width: calc(100% - 60px);
            transition: margin-left 0.3s ease;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.2);
        }
        .sidebar:hover ~ .main {
            margin-left: 250px;
            width: calc(100% - 250px);
        }
        .navbar {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            background-color: rgba(34, 34, 34, 0.9);
            color: white;
            padding: 12px 20px;
            position: sticky;
            top: 0;
            z-index: 1;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            border-radius: 10px;
            margin-bottom: 20px;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            padding: 12px 20px;
            font-size: 18px;
            transition: background-color 0.3s ease;
        }
        .navbar a:hover {
            background-color: #ff5733;
            border-radius: 10px;
        }
    </style>
</head>
<body>
    <div class="sidebar">
        <div class="logo">
            <img src="/static/logo cgai 2.jpg" alt="Logo">
            <h1>Comix GenAI</h1>
        </div>
        <button onclick="location.href='/generate-story'"><span>Generate a Story</span></button>
        <button onclick="location.href='/generate-images'"><span>Generate Images</span></button>
        <button onclick="location.href='/generate-comic'"><span>Generate Comic</span></button>
        <button onclick="location.href='/image-colorization'"><span>Image Colorization</span></button>
    </div>

    <div class="main">
        <div class="navbar">
            <a href="/">Home</a>
            <a href="/history">History</a>
            <a href="/about">About Us</a>
            <a href="/contact">Contact Us</a>
        </div>

        <h1>Your Generated Comic</h1>
    <div>
        {% for panel_url in panel_urls %}
            <img src="{{ panel_url }}" alt="Comic Panel" style="margin: 10px; border: 1px solid #000; max-width: 100%; height: auto;">
        {% endfor %}
    </div>
    <a href="{{ url_for('generate_comic') }}">Generate Another Comic</a>
    </div>
</body>
</html>
"""

# Save the HTML file
with open("/content/static/view_comic.html", "w") as html_file:
    html_file.write(view_comic_html)